# Usando ML para resolver problemas de negocio
### Favio Vázquez

## El problema: Churn de clientes

![](https://www.insideselfstorage.com/sites/insideselfstorage.com/files/styles/article_featured_retina/public/Sad-Customer-Service.jpg?itok=S9sd0R3T)
Crédito: https: //www.insideselfstorage.com/customer-service/7-deadly-customer-service-situations-self-storage-and-how-handle-them

El churn de clientes se define como cuando los clientes o suscriptores dejan de hacer negocios con una empresa o servicio.

Cada fila representa un cliente, cada columna contiene los atributos del cliente.

El conjunto de datos incluye información sobre:

- Clientes que se fueron en el último mes: la columna se llama **Churn**
- Servicios para los que se ha suscrito cada cliente: teléfono, varias líneas, Internet, seguridad en línea, respaldo en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas
- Información de la cuenta del cliente: cuánto tiempo ha sido cliente, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales.
- Información demográfica sobre los clientes: sexo, rango de edad y si tienen pareja y dependientes

## Comprender el contexto y el problema empresarial

Antes de dedicar tiempo a intentar resolver un problema comercial, debemos estar seguros de que tenemos un problema. Para eso necesitamos tener reuniones con las personas cercanas al problema empresarial y los carniceros.

Tuvimos dos reuniones, una con el área comercial y otra con los principales ejecutivos. Esto es lo que escuchamos:

- Los clientes se van pero no sabemos por qué.
- Tenemos 1 mes de datos de clientes donde sabemos cuáles se quedaron y cuáles se fueron.
- La rotación de clientes no puede superar el 15% mensual debido a nuestros cálculos.
- No conocemos el impacto financiero de perder un cliente.
- Podemos dar un vale por USD \$ 500.
- El valor de vida útil estimado para un cliente es USD \$ 7500.

Tras esas reuniones tenemos que comprobar los datos existentes en la empresa y encontrar información útil en ella. Supongamos que lo hicimos y después de un proceso de integración de datos creamos un conjunto de datos completo para nuestros clientes y su información. Recuerde que estamos trabajando con una empresa de telecomunicaciones.

## Bibliotecas

In [ ]:
!pip install datatable

In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import datatable as dt
from datatable import f, min, max, mean
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
import scipy.stats as stats
import warnings
import numpy as np
warnings.filterwarnings("ignore")

## Cargar datos

In [ ]:
df = dt.fread("data/churn-data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

El color significa el tipo de datos donde el rojo denota una cadena, el verde denota int y el azul significa flotante.

## ¿Cuántos clientes se han ido?

In [ ]:
df[f.Churn == "Yes", dt.count()]

In [ ]:
1869/7043

Se han ido 1869 clientes, es decir, el 26% de nuestros clientes. Entonces, si recordamos las métricas del negocio, tenemos un problema.

## ¿Cuánto dinero hemos perdido debido a la pérdida de clientes?

In [ ]:
df[:, dt.count(), dt.by(dt.f.Churn)]

In [ ]:
df[dt.f.Churn == 'Yes', 'TotalCharges'].sum1()

Hemos perdido $ 2.862.926 debido a la pérdida de clientes. Intentemos resolver este problema.

## Exploración de datos

In [ ]:
df_pandas = df.to_pandas()

In [ ]:
df_pandas.head()

In [ ]:
def diagnostic_plots(df_pandas, variable):
    
    plt.figure(figsize=(20, 9))

    plt.subplot(1, 3, 1)
    sns.histplot(data = df_pandas, x=variable, bins=30, kde=True)
    plt.title('Histogram')
    
    plt.subplot(1, 3, 2)
    stats.probplot(df_pandas[variable], dist="norm", plot=plt)
    plt.ylabel('RM quantiles')

    plt.subplot(1, 3, 3)
    sns.boxplot(x=df_pandas[variable])
    plt.title('Boxplot')
    
    plt.show()

In [ ]:
num_columns=df_pandas.select_dtypes(include=["number"]).columns
num_columns

In [ ]:
for i in num_columns:
    diagnostic_plots(df_pandas,i)

In [ ]:
sns.pairplot(df_pandas.drop("SeniorCitizen",axis=1),hue="Churn",aspect=3);

In [ ]:
fig = px.histogram(df_pandas, x="Churn")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.histogram(df_pandas, x="Churn", color="SeniorCitizen")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.histogram(df_pandas, x="Churn", color="OnlineSecurity", barmode="group")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.box(df_pandas, x='Churn', y = 'tenure')
fig.show()

In [ ]:
ax = sns.kdeplot(df_pandas.MonthlyCharges[(df_pandas["Churn"] == 'No') ],
                color="Red", shade = True);
ax = sns.kdeplot(df_pandas.MonthlyCharges[(df_pandas["Churn"] == 'Yes') ],
                ax =ax, color="Blue", shade= True);
ax.legend(["Not Churn","Churn"],loc='upper right');
ax.set_ylabel('Density');
ax.set_xlabel('Monthly Charges');
ax.set_title('Distribution of monthly charges by churn');

In [ ]:
corr = df_pandas.apply(lambda x: pd.factorize(x)[0]).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

heat = go.Heatmap(
    z=corr.mask(mask),
    x=corr.columns,
    y=corr.columns,
    colorscale=px.colors.diverging.RdBu,
    zmin=-1,
    zmax=1
)

pio.templates.default = "plotly_white"


fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='Heatmap', 
    title_x=0.5, 
    width=1000, 
    height=1000,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

fig=go.Figure(data=[heat])
fig.show()

## Limpieza de datos

In [ ]:
df.names

In [ ]:
df.stypes

In [ ]:
## missing values
dt.math.isna(df).sum()

Solo tenemos 11 valores faltantes en la columna TotalCharges.

In [ ]:
## Delete missing rows
df = df[dt.rowall(dt.f[:] != None), :]

In [ ]:
# Delete customerID
del df[:, "customerID"]

In [ ]:
df.head()

In [ ]:
# Enconde Churn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[:, 'Churn'] = dt.Frame(le.fit_transform(np.ravel(df[:, 'Churn'])))

In [ ]:
# Function for OHE
def ohe_columns(columns,df):
    df_work = df.copy()
    for column in columns:
        df_ohe = dt.str.split_into_nhot(df_work[column])
        df_ohe.names = [f'{column}_{col}' for col in df_ohe.names]
        df_work.cbind(df_ohe)
    return df_work

In [ ]:
# Select categorical columns
categorical_columns = df[:, str].names

In [ ]:
# Get final df after OHE
df_final = ohe_columns(categorical_columns,df)

In [ ]:
# Delete orignal columns
del df_final[:, categorical_columns]

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv("data/churn_data_cleaned.csv")

## Modelado

In [ ]:
!pip install h2o

In [ ]:
import h2o

In [ ]:
h2o.init()

### AutoML con H2O

In [ ]:
from h2o.automl import *

In [ ]:
train, test = dataset.split_frame([0.8], seed=42)

In [ ]:
print("train:%d test:%d" % (train.nrows, test.nrows))

In [ ]:
# Identify predictors and response
x = train.columns
y = "Churn"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
aml = H2OAutoML(max_runtime_secs = 90, 
                max_models = 25,  
                seed = 42, 
                project_name='classification_1',
                sort_metric = "AUC")

%time aml.train(x = x, y = y, training_frame = train)

In [ ]:
lb = aml.leaderboard
lb.head(rows = lb.nrows)

In [ ]:
aml.leader

In [ ]:
aml.leader.model_performance(test_data=test)

In [ ]:
aml.leader.model_performance(test_data=test).plot()

In [ ]:
aml.explain(test)

In [ ]:
aml.leader.model_performance(test_data=test).confusion_matrix()

### GBM con H2O

In [ ]:
dataset = h2o.import_file("data/churn_data_cleaned.csv")

In [ ]:
dataset.head()

In [ ]:
from h2o.estimators import *
from h2o.grid import *

In [ ]:
train, valid, test = dataset.split_frame([0.7, 0.15], seed=42)

In [ ]:
# Identify predictors and response
x = train.columns
y = "Churn"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
valid[y] = valid[y].asfactor()

In [ ]:
gbm = H2OGradientBoostingEstimator(seed = 42, 
                                   model_id = 'default_gbm')

%time gbm.train(x = x, y = y, training_frame = train, validation_frame = valid)

In [ ]:
gbm

In [ ]:
gbm.predict(valid)

In [ ]:
default_gbm_per = gbm.model_performance(test)

In [ ]:
default_gbm_per

In [ ]:
# Hyperparameter estimation

gbm = H2OGradientBoostingEstimator(ntrees = 500,
                                   learn_rate = 0.05,
                                   seed = 42,
                                   model_id = 'grid_gbm')

hyper_params_tune = {'max_depth' : [4, 5, 6, 7, 8],
                     'sample_rate': [x/100. for x in range(20,101)],
                     'col_sample_rate' : [x/100. for x in range(20,101)],
                     'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                     'col_sample_rate_change_per_level': [x/100. for x in range(90,111)]}

search_criteria_tune = {'strategy': "RandomDiscrete",
                        'max_runtime_secs': 90,  
                        'max_models': 100,  ## build no more than 100 models
                        'seed' : 42}

random_grid = H2OGridSearch(model = gbm, hyper_params = hyper_params_tune,
                            grid_id = 'random_grid',
                            search_criteria = search_criteria_tune)

%time random_grid.train(x = x, y = y, training_frame = train, validation_frame = valid)

In [ ]:
sorted_random_search = random_grid.get_grid(sort_by = 'auc',decreasing = True)
sorted_random_search.sorted_metric_table()

In [ ]:
tuned_gbm = sorted_random_search.models[0]

In [ ]:
tuned_gbm_per = tuned_gbm.model_performance(test)
print(tuned_gbm_per.auc())

In [ ]:
tuned_gbm.explain(test)

In [ ]:
tuned_gbm.explain_row(test, row_index=15)

In [ ]:
tuned_gbm.model_performance(test_data=dataset).confusion_matrix()

In [ ]:
dataset.nrows

Esta matriz de confusión es para todo el conjunto que incluye el 100% de nuestros datos (7032 filas). Tenemos X verdaderos positivos (X%): estos son los clientes para los que podremos ampliar el valor de vida útil. Si no hubiéramos predicho, entonces no había oportunidad de intervenir.

También tenemos X (X%) falsos positivos donde perderemos dinero porque la promoción ofrecida a estos clientes solo será un costo adicional.

X (X%) son verdaderos negativos (buenos clientes) y X (X%) son falsos negativos (esta es una oportunidad perdida).

En un modelo de churn, a menudo la recompensa de los verdaderos positivos es muy diferente al costo de los falsos positivos. Usemos las siguientes suposiciones:

- Se ofrecerá un cupón de \$500 a todos los clientes identificados como Churn (Verdadero Positivo + Falso Positivo)
- Si podemos detener la deserción, ganaremos \$7500 en valor de por vida para el cliente.

| Descripción                    | Clientes  | Valor | Total     |
|--------------------------------|-----------|-------|-----------|
| True Positive                  | X      | 7500   | X |
| True Positive + False Positive | X      | 500  | X  |
|                                |           |       | **X** |

### Referencias

- https://github.com/vopani/datatableton
- https://docs.h2o.ai/h2o/latest-stable/h2o-docs/welcome.html
- https://h2oai.github.io/tutorials/introduction-to-machine-learning-with-h2o-3-automl/#0
- https://www.kaggle.com/bhartiprasad17/customer-churn-prediction
- https://www.kaggle.com/parulpandey/speed-up-your-data-munging-with-python-s-datatable/data
- https://www.kaggle.com/ferhatmetin34/telco-churn-prediction-under-oversampling-automl/data
- https://www.kaggle.com/sudalairajkumar/hyperparam-tuning-automl
- https://www.kaggle.com/nishantdhingra/h2o-automl-kfold
- https://towardsdatascience.com/predict-customer-churn-the-right-way-using-pycaret-8ba6541608ac
- https://github.com/h2oai/h2o-meetups/blob/master/2021_02_26_USFData_H2OAutoMLExplain/h2o_automl_explain_usfca_feb2021.pdf
- https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d